In [15]:
import os
import sklearn
import pandas as pd
import csv

In [16]:
n_gram_value = 5

In [17]:
def get_ngrams(input_list, n):    
    return zip(*[input_list[i:] for i in range(n)])


In [18]:
def getAllPossibleWords(file_path): 
    possible_words=[]
    with open(file_path) as f:
        for line in f:
            for word in line.split():
                possible_words.append(word)     
   # possible_words = map(lambda x : unidecode.unidecode(x),possible_words)
    for c in ['`',"[","]","(",")","\"",".","''",","]:
        possible_words = map(lambda x: x.replace(c,""),possible_words)
        
    possible_words = map(lambda x: x.replace('&','And'),possible_words)
    #possible_words = map(lambda x: x.replace('of','Of'),possible_words)
    possible_words=filter(lambda x:len(x)>0,possible_words)    
    
    return possible_words

In [22]:
def getListFromFiles(filename):
    file_path = os.path.join(os.getcwd(),filename)
    info_list = []
    with open(file_path) as f:
        for line in f:
            for word in line.split():
                info_list.append(word)
    return info_list

In [23]:
def writeToCsv(processed_list):
    with open("output-sreejita1.csv",'wb') as resultFile:
        wr = csv.writer(resultFile, dialect='excel')
        for row in processed_list:
            wr.writerows(row)

In [24]:
def preprocessing(possible_words,doc_id):    
    suffix_list = getListFromFiles("company-suffixes.txt")
    prefix_list = getListFromFiles("company-prefixes.txt")    
    common_words = getListFromFiles("common-words.txt")
    preprocessed_list =[]    
    marked_list=[]
    word_number=len(possible_words)
    for i in range(5,0,-1):
        word_count=0
        ngram_list = get_ngrams(possible_words,i)
        for ngram in ngram_list:            
            #print ngram   
            
            #if markup is both at the beginning or at the end of the word group accept and label as 1
            if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1):
                preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,1])
            
            #get all instances of format <markup>Microsoft</markup>
            if(len(ngram)==1 and '</markup>' in ngram[0] and '<markup>' in ngram[0]):
                preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,1])            
                    
            else:
                #prune away all n grams with non uppercase first character
                if(all(word[0].isupper() and "markup" not in word for word in ngram)):       
                    
                    # prune common words
                    if(any(word not in common_words for word in ngram)):                        
                        preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])

                    #prune unigrams 
                    elif(i==1 and word_number> word_count+1):                        
                        
                        #add the unigram to the list if the next word begins with lower case
                        if(possible_words[word_count+1][0].islower()):
                            preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])  
            word_count = word_count+1
    return preprocessed_list    

In [25]:
base_directory = os.path.dirname(os.getcwd())
data_directory = os.path.join(base_directory,"Dataset","Sreejita")
file_list = os.listdir(data_directory)
possible_words=[]
processed_list=[]
count=0
doc_id = 101
for file in file_list:
    #doc_id = file[0:3]
    #print(doc_id)
    #if(int(doc_id)!=95):
    #    continue    
    possible_words = getAllPossibleWords(os.path.join(data_directory,file))     
    
    #start from here .. processessed list has the structure[[word,doc_id,word_count_start,word_count_end,label]]
    processed_list.append(preprocessing(possible_words,doc_id))
    doc_id += 1
    #writeToCsv(processed_list)   
  

In [26]:
processed_list



[[['Chief Executive Officer Robert Gannicott', 101, 108, 112, 0],
  ['Executive Officer Robert Gannicott Gannicott', 101, 109, 113, 0],
  ['Chief Executive Officer Robert', 101, 108, 111, 0],
  ['Executive Officer Robert Gannicott', 101, 109, 112, 0],
  ['Officer Robert Gannicott Gannicott', 101, 110, 113, 0],
  ['<markup>Aber Diamond Corp</markup>', 101, 0, 2, 1],
  ['Toronto Stock Exchange', 101, 18, 20, 0],
  ['<markup>Rio Tinto Plc</markup>', 101, 94, 96, 1],
  ['Chief Executive Officer', 101, 108, 110, 0],
  ['Executive Officer Robert', 101, 109, 111, 0],
  ['Officer Robert Gannicott', 101, 110, 112, 0],
  ['Robert Gannicott Gannicott', 101, 111, 113, 0],
  ['Harry Winston', 101, 7, 8, 0],
  ['Toronto Stock', 101, 18, 19, 0],
  ['Stock Exchange', 101, 19, 20, 0],
  ['Northwest Territories', 101, 81, 82, 0],
  ['Canadian Press', 101, 103, 104, 0],
  ['Chief Executive', 101, 108, 109, 0],
  ['Executive Officer', 101, 109, 110, 0],
  ['Officer Robert', 101, 110, 111, 0],
  ['Robert G

In [27]:
ngram = ('<markup>Gazprom', 'Lukoil</markup>')
if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1):
    print "hallo"


hallo


In [28]:
["markup" in word for word in ngram].count(True)==2

True

In [47]:
import pycountry
country_names = []
for country in pycountry.countries:
    country_names.append(country.name)
    if hasattr(country,'common_name'):
        country_names.append(country.common_name)
    if hasattr(country, 'official_name'):
        country_names.append(country.official_name)

country_names.append("America")
country_names.append("Korea")
country_names.append("South Korea")
country_names.append("North Korea")
country_names.append("U.S.")

if "USA" in country_names:
    print "YES"
    
continent_list = ['Asia', 'Africa', 'South America', 'North America', 'Antarctica', 'Europe', 'Australia']

In [48]:
cities = []
with open('world-cities.csv', 'r') as inp:
        for row in csv.reader(inp):
            if(row and row[3] != "" and row[0]!="name"):
                cities.append(row[0])

cities.append('Bombay')
print cities

['les Escaldes', 'Andorra la Vella', 'Umm al Qaywayn', 'Ras al-Khaimah', 'Khawr Fakk\xc4\x81n', 'Dubai', 'Dibba Al-Fujairah', 'Dibba Al-Hisn', 'Sharjah', 'Ar Ruways', 'Al Fujayrah', 'Al Ain', 'Ajman', 'Adh Dhayd', 'Abu Dhabi', 'Zaranj', 'Taloqan', 'Sh\xc4\xabn\xe1\xb8\x8fan\xe1\xb8\x8f', 'Shibirgh\xc4\x81n', 'Shahrak', 'Sar-e Pul', 'Sang-e Ch\xc4\x81rak', 'A\xc4\xabbak', 'Rust\xc4\x81q', 'Qarq\xc4\xabn', 'Qar\xc4\x81wul', 'Pul-e Khumr\xc4\xab', 'Paghm\xc4\x81n', 'Nahr\xc4\xabn', 'Maymana', 'Mehtar L\xc4\x81m', 'Maz\xc4\x81r-e Shar\xc4\xabf', 'Lashkar G\xc4\x81h', 'Kushk', 'Kunduz', 'Kh\xc5\x8dst', 'Khulm', 'Kh\xc4\x81sh', 'Khanabad', 'Karukh', 'Kandah\xc4\x81r', 'Kabul', 'Jal\xc4\x81l\xc4\x81b\xc4\x81d', 'Jabal os Saraj', 'Her\xc4\x81t', 'Ghormach', 'Ghazni', 'Gereshk', 'Gard\xc4\x93z', 'Fayzabad', 'Farah', 'Kafir Qala', 'Charikar', 'Barak\xc4\xab Barak', 'B\xc4\x81my\xc4\x81n', 'Balkh', 'Baghl\xc4\x81n', '\xc4\x80rt Khw\xc4\x81jah', '\xc4\x80sm\xc4\x81r', 'Asad\xc4\x81b\xc4\x81d', 'An

In [65]:
desig = open('designations.txt').read().split('\n')
desig_lower = [x.lower() for x in desig]
print desig_lower

['chief executive officer', 'chief financial officer', 'co-chief executive', 'chief investment officer', 'analyst', 'ceo', 'cio', 'cto', 'cfo', 'manager', 'director', 'president', 'vice-president', 'vice president', 'chief', 'executive', 'officer', 'officers', 'analysts', 'managers', 'directors', 'co-chief', 'co chief']


In [107]:
stock_x = open('stock-exchange-list.txt').read().split('\n')
print stock_x
unigram_prune = open('unigram-prune-neg.txt').read().split('\n')
prune = open('pruning-neg.txt').read().split('\n') 
prune_lower = [x.lower() for x in prune]
prune_lower

['New York Stock Exchange', 'NASDAQ', 'Japan Exchange Group', 'Shanghai Stock Exchange', 'Euronext', 'London Stock Exchange', 'Hong Kong Stock Exchange', 'Shenzhen Stock Exchange', 'TMX Group', 'National Stock Exchange of India', 'Deutsche B\xc3\xb6rse', 'Bombay Stock Exchange', 'Korea Exchange', 'SIX Swiss Exchange', 'Nasdaq Nordic', 'Australian Securities Exchange', 'JSE Limited', 'Taiwan Stock Exchange', 'BM&F Bovespa', 'BME Spanish Exchanges', 'Toronto Stock Exchange']


['index',
 'authority',
 'exchange',
 'department',
 'committee',
 'commission',
 'bloomberg',
 'bureau',
 'news',
 'agency',
 'headquarters',
 'nasa']

In [128]:
pruned_list = []
for doc in processed_list:
    #print doc
    for inst in doc:
        if inst[4] == 0:
            if inst[0] in country_names or inst[0] in continent_list or inst[0] in cities or inst[0] in stock_x:
                pruned_list.append(inst)
            elif inst[0].endswith('n'):
                str = inst[0][:-1]
                if str in country_names or str in continent_list:
                    pruned_list.append(inst)
            elif "based" in inst[0]:
                pruned_list.append(inst)
            elif any(d in inst[0].lower().split() for d in desig_lower):
                pruned_list.append(inst)
            elif any(p in inst[0].lower().split() for p in prune_lower):
                pruned_list.append(inst)
            elif '(' in inst[0] or ')' in inst[0]:
                pruned_list.append(inst)
            elif len(inst[0].split()) == 1:
                if len(inst[0]) == 1:
                    pruned_list.append(inst)
                elif inst[0] in unigram_prune:
                    pruned_list.append(inst)
                elif inst[0] == inst[0].upper():
                    pruned_list.append(inst)
                
                
                
            
#for p in pruned_list:
 #   print p
    
print len(pruned_list)

1808
